In [1]:
import os, glob
import subprocess
import tempfile
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
from rioxarray import merge
from osgeo import gdal
from joblib import Parallel, delayed
from oggm import utils
import gzip
import json
import topojson
import shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely.wkt import loads, dumps

In [8]:
# RGI
#version = '62'
version = '70G'
OUT_PATH = f"/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI{version}/geometries"

In [ ]:
list_gdfs = []

for rgi in range(1,20):

    FILE_SHP_RGI = utils.get_rgi_region_file(region=f"{rgi:02d}", version=version)
    print(rgi, FILE_SHP_RGI)

    gdf = gpd.read_file(FILE_SHP_RGI)

    gdf = gdf[['geometry']]

    gdf_rounded = gdf.copy()

    gdf_rounded['geometry'] = gdf_rounded['geometry'].apply(lambda geom: shapely.set_precision(geom, 0.00001))

    list_gdfs.append(gdf_rounded)

print('Combining geodataframes')
combined_gdf = pd.concat(list_gdfs, ignore_index=True)
print('Done.')

print('Creating geojson')
geojson_file = f"{OUT_PATH}/all_geometries_rgi{version}_cleaned.geojson"
print(OUT_PATH)
print(geojson_file)

combined_gdf.to_file(geojson_file, driver='GeoJSON')

tippecanoe_path_out = f"/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI{version}/geometries/"
tippecanoe_file_out = f"geoms_rgi{version}.mbtiles"
tippecanoe_out_layer_name = f"geoms_rgi{version}"
tippecanoe_out = tippecanoe_path_out + tippecanoe_file_out

# Construct the Tippecanoe command
tippecanoe_command = f"tippecanoe -o {tippecanoe_out} -l {tippecanoe_out_layer_name} -zg {geojson_file} --force"

# Run Tippecanoe command via subprocess
subprocess.run(tippecanoe_command, shell=True, check=True)

# remove geojson_file
os.remove(geojson_file)
print("Deleted: ", geojson_file)

In [15]:
# Antarctic Peninsula

In [14]:
PENINSULA_PATH = f"/media/maffe/nvme/Antarctic_peninsula_geometries/final_product/"

peninsula = gpd.read_file(PENINSULA_PATH+"antarctic_peninsula.shp")

peninsula = peninsula[['geometry']]

peninsula_rounded = peninsula.copy()

peninsula_rounded['geometry'] = peninsula_rounded['geometry'].apply(lambda geom: shapely.set_precision(geom, 0.00001))

print('Creating geojson')
geojson_file = f"{PENINSULA_PATH}peninsula_cleaned.geojson"
print(PENINSULA_PATH)
print(geojson_file)

peninsula_rounded.to_file(geojson_file, driver='GeoJSON')

tippecanoe_path_out = f"{PENINSULA_PATH}"
tippecanoe_file_out = f"peninsula.mbtiles"
tippecanoe_out_layer_name = f"peninsula"
tippecanoe_out = tippecanoe_path_out + tippecanoe_file_out

# Construct the Tippecanoe command
tippecanoe_command = f"tippecanoe -o {tippecanoe_out} -l {tippecanoe_out_layer_name} -zg {geojson_file} --force"

# Run Tippecanoe command via subprocess
subprocess.run(tippecanoe_command, shell=True, check=True)

# remove geojson_file
os.remove(geojson_file)
print("Deleted: ", geojson_file)

Creating geojson
/media/maffe/nvme/Antarctic_peninsula_geometries/final_product/
/media/maffe/nvme/Antarctic_peninsula_geometries/final_product/peninsula_cleaned.geojson


/media/maffe/nvme/Antarctic_peninsula_geometries/final_product/peninsula_cleaned.geojson:6: ignoring dimensions beyond two
In JSON object [-65.53776,-69.10115,0.0]
In JSON object {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-65.53776,-69.10115,0.0],[-65.13818,-69.57178,0.0],[-65.12908,-69.97426,0.0],[-65.13762,-69.97499,0.0],[-65.14062,-69.97524,0.0],[-65.15664,-69.97497,0.0],[-65.18098,-69.97296,0.0],[-65.20441,-69.97105,0.0],[-65.21229,-69.971,0.0],[-65.22921,-69.97089,0.0],[-65.24442,-69.97176,0.0],[-65.26654,-69.97526,0.0],[-65.30536,-69.9824,0.0],[-65.3194,-69.98461,0.0],[-65.32319,-69.98521,0.0],[-65.33975,-69.9873,0.0],[-65.35...
23 features, 1999786 bytes of geometry, 39 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z0 for features about 826875 feet (252032 meters) apart
Choosing a maxzoom of -z9 for resolution of about 533 feet (162 meters) within features


Deleted:  /media/maffe/nvme/Antarctic_peninsula_geometries/final_product/peninsula_cleaned.geojson


  99.9%  9/162/382  
